# Optimizer Example

This notebook demonstrates how to use the PyQuOpt numerical optimization code to generate novel implementations of a specified unitary matrix. In this case, the Toffoli gate is explored.

### Imports

In [1]:
import sys
import numpy as np

sys.path.append('..')
from pyquopt import *

### Optimization Hyperparameters

In [2]:
num_qubits = 3
num_mq_instructions = 6  # want to explore circuits with six multi-qubit gates
num_params = 3 * num_qubits * (num_mq_instructions + 1)  # formula explained in MICRO paper

mq_dict = {
    0: ThreeGates.CX01,
    1: ThreeGates.CX02,
    2: ThreeGates.CX12,
}

In [3]:
alpha = 0.30  # penalty for non-standard angles
gamma = 0   # penalty for large angles
non_fixed_params = np.ones(num_params)  # leave all parameters unfixed
fixed_params_vals = np.zeros(num_params)  # unfixed parameters should have a "fixed" value of 0

In [7]:
mq_instructions = [0, 2, 1, 2, 1, 0]

### Instantiate and Run Optimizer

In [5]:
optimizer = Optimizer(num_qubits=num_qubits, mq_instructions=mq_instructions, mq_dict=mq_dict, target=ThreeGates.TOFFOLI,
                     alpha=alpha, gamma=gamma, non_fixed_params=non_fixed_params, fixed_params_vals=fixed_params_vals)

In [6]:
opt_params, opt_val = optimizer.find_parameters_least_squares(20)

/Users/revilooliver/Documents/quantum_computing/anaconda/anaconda3/envs/qiskit-terra/lib/python3.7/site-packages/scipy/optimize/_lsq/least_squares.py:112: UserWarning: Setting `ftol` below the machine epsilon (2.22e-16) effectively disables the corresponding termination condition.
  .format(name, EPS))


KeyboardInterrupt: 

Note that there is also a function `optimizer.find_parameters_least_squares_par(num_guesses, num_procs)` that allows for process-based parallelism.

In [7]:
opt_val

0.017593700031446932

In [8]:
opt_params_rounded = round_params(opt_params)
print(opt_params_rounded)

[270.  80. 315. 270.  45. 180. 310.  45. 240.  45. 135. 225.   0. 135.
 225. 240. 240.  10.  90. 225. 165.  60. 180. 315. 355.  10. 120.   0.
 210. 300. 330. 225. 135. 135. 225.  90. 180. 165.  30. 120. 315. 150.
 260.  45. 240. 315.  30.  60. 300.  30. 345.  60. 315.  45. 315. 120.
 300. 180. 225.  30. 315. 335. 305.]


### Verify Implementation Accuracy

In [14]:
ub = UnitaryBuilder(num_qubits, mq_instructions, mq_dict)

In [15]:
implementation_matrix = ub.build_unitary(opt_params_rounded * np.pi / 180)

In [16]:
print(get_unitary_infidelity(ThreeGates.TOFFOLI, implementation_matrix, 8))

0.28016257386700694


An infidelity less than 0.01 is acceptable.

In [17]:
params_list = []
for i in range(0,3):
    for j in range(0,3):
        for k in range(0,3):
            for l in range(0,3):
                temp = [0] + [i] + [j] + [k] + [l] + [0]
                print(temp)
                params_list.append(temp)

[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 0]
[0, 0, 0, 0, 2, 0]
[0, 0, 0, 1, 0, 0]
[0, 0, 0, 1, 1, 0]
[0, 0, 0, 1, 2, 0]
[0, 0, 0, 2, 0, 0]
[0, 0, 0, 2, 1, 0]
[0, 0, 0, 2, 2, 0]
[0, 0, 1, 0, 0, 0]
[0, 0, 1, 0, 1, 0]
[0, 0, 1, 0, 2, 0]
[0, 0, 1, 1, 0, 0]
[0, 0, 1, 1, 1, 0]
[0, 0, 1, 1, 2, 0]
[0, 0, 1, 2, 0, 0]
[0, 0, 1, 2, 1, 0]
[0, 0, 1, 2, 2, 0]
[0, 0, 2, 0, 0, 0]
[0, 0, 2, 0, 1, 0]
[0, 0, 2, 0, 2, 0]
[0, 0, 2, 1, 0, 0]
[0, 0, 2, 1, 1, 0]
[0, 0, 2, 1, 2, 0]
[0, 0, 2, 2, 0, 0]
[0, 0, 2, 2, 1, 0]
[0, 0, 2, 2, 2, 0]
[0, 1, 0, 0, 0, 0]
[0, 1, 0, 0, 1, 0]
[0, 1, 0, 0, 2, 0]
[0, 1, 0, 1, 0, 0]
[0, 1, 0, 1, 1, 0]
[0, 1, 0, 1, 2, 0]
[0, 1, 0, 2, 0, 0]
[0, 1, 0, 2, 1, 0]
[0, 1, 0, 2, 2, 0]
[0, 1, 1, 0, 0, 0]
[0, 1, 1, 0, 1, 0]
[0, 1, 1, 0, 2, 0]
[0, 1, 1, 1, 0, 0]
[0, 1, 1, 1, 1, 0]
[0, 1, 1, 1, 2, 0]
[0, 1, 1, 2, 0, 0]
[0, 1, 1, 2, 1, 0]
[0, 1, 1, 2, 2, 0]
[0, 1, 2, 0, 0, 0]
[0, 1, 2, 0, 1, 0]
[0, 1, 2, 0, 2, 0]
[0, 1, 2, 1, 0, 0]
[0, 1, 2, 1, 1, 0]
[0, 1, 2, 1, 2, 0]
[0, 1, 2, 2, 0, 0]
[0, 1, 2, 2,

In [18]:
useful_params = []
useful_bitstring = []
for param in params_list[::-1]:
    mq_instructions = param
    print(mq_instructions)
    optimizer = Optimizer(num_qubits=num_qubits, mq_instructions=mq_instructions, mq_dict=mq_dict, target=ThreeGates.TOFFOLI,
                     alpha=alpha, gamma=gamma, non_fixed_params=non_fixed_params, fixed_params_vals=fixed_params_vals)
    opt_params, opt_val = optimizer.find_parameters_least_squares(20)
    opt_params_rounded = round_params(opt_params)
    implementation_matrix = ub.build_unitary(opt_params)
    infidelity = get_unitary_infidelity(ThreeGates.TOFFOLI, implementation_matrix, 8)
    print(infidelity)
    if infidelity < 0.1:
        useful_params.append(opt_params_rounded)
        useful_bitstring.append(non_fixed_params)

[0, 2, 2, 2, 2, 0]
0.2720361820798538
[0, 2, 2, 2, 1, 0]
0.8000975365360785
[0, 2, 2, 2, 0, 0]
0.9639954649750233
[0, 2, 2, 1, 2, 0]
0.8157895789083678
[0, 2, 2, 1, 1, 0]
0.9438172752823057
[0, 2, 2, 1, 0, 0]
0.9954184447625989
[0, 2, 2, 0, 2, 0]
0.9429229289015723
[0, 2, 2, 0, 1, 0]
0.9857984892369451
[0, 2, 2, 0, 0, 0]
0.6350081793086884
[0, 2, 1, 2, 2, 0]
0.7955735131016979
[0, 2, 1, 2, 1, 0]
0.8150056334349298
[0, 2, 1, 2, 0, 0]
0.9869011688464315
[0, 2, 1, 1, 2, 0]
0.9488572775346239
[0, 2, 1, 1, 1, 0]
0.977712855260661
[0, 2, 1, 1, 0, 0]
0.9547743544893714
[0, 2, 1, 0, 2, 0]
0.9953063889800732
[0, 2, 1, 0, 1, 0]
0.9423593903563479
[0, 2, 1, 0, 0, 0]
0.9974374093232193
[0, 2, 0, 2, 2, 0]
0.9347907005030955
[0, 2, 0, 2, 1, 0]
0.9908050590246616
[0, 2, 0, 2, 0, 0]
0.9802516006443867
[0, 2, 0, 1, 2, 0]
0.9877249224710646
[0, 2, 0, 1, 1, 0]
0.991099007383778
[0, 2, 0, 1, 0, 0]
0.9504480402727898
[0, 2, 0, 0, 2, 0]
0.9754104552527268
[0, 2, 0, 0, 1, 0]
0.9592499439416766
[0, 2, 0, 0, 0

In [ ]:
021020